In [1]:
MAIN_DIR = "C:\\Users\\gyujang95\\Desktop\\Marat\\FIPGraph"

### textures, indices to be processed

In [2]:
textures = {
    '30': {'sve_indices': list(range(200))},
}

# preprocessing

### hdf5s to nx graphs

In [3]:
from utils_preprocessing import write_nx_graph

for texture, info in textures.items():
    
    sve_indices = info['sve_indices']

    for ith_sve in sve_indices:
        write_nx_graph(f'{MAIN_DIR}\\preprocessing\\graph_sves', f'{MAIN_DIR}\\data\\hdf5s_SVE', texture, ith_sve)

KeyboardInterrupt: 

### fip hdf5s to csv files

In [ ]:
from utils_preprocessing import write_fip_table

for texture, info in textures.items():
    
    sve_indices = info['sve_indices']

    for ith_sve in sve_indices:
        write_fip_table(f'{MAIN_DIR}\\preprocessing\\fiptables\\{texture}', f'{MAIN_DIR}\\data\\hdf5s_FIP', texture, ith_sve)

### Make scalers

In [ ]:
from utils_preprocessing import make_scaler

for texture in textures:
    make_scaler(scaler_dir=f'{MAIN_DIR}\\preprocessing\\scalers',
                graph_dir=f'{MAIN_DIR}\\preprocessing\\graph_sves', 
                fip_dir=f'{MAIN_DIR}\\preprocessing\\FIPtables', 
                texture=texture, 
                num_nfeat=17, 
                num_efeat=1)

c:\Users\gyujang95\AppData\Local\anaconda3\envs\env_Marat_20231026_2\lib\site-packages\torch_geometric\utils\convert.py:260: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:264.)
  data[key] = torch.tensor(value)


### Make train/validation data lists

In [ ]:
from utils_preprocessing import make_torchdataset

for texture in textures:
    make_torchdataset(torchdata_dir=f'{MAIN_DIR}\\preprocessing\\graph_torchdata', 
                      graph_dir=f'{MAIN_DIR}\\preprocessing\\graph_sves', 
                      fip_dir=f'{MAIN_DIR}\\preprocessing\\FIPtables',
                      scaler_dir=f'{MAIN_DIR}\\preprocessing\\scalers', 
                      texture=texture, 
                      train_fraction=0.9, 
                      seed=42)

ImportError: cannot import name 'make_torchdataset' from 'utils_preprocessing' (c:\Users\gyujang95\Desktop\Marat\FIPGraph\code\utils_preprocessing.py)